In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [2]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import time
from xgboost import XGBRegressor
import torch
import os
import sys
import psutil
from skforecast.recursive import ForecasterRecursive

In [5]:
# Print information abput the GPU and CPU
# ==============================================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0) / 1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0) / 1024**3, 1), 'GB')

print(f"CPU RAM Free: {psutil.virtual_memory().available / 1024**3:.2f} GB")

Using device: cuda
NVIDIA T1200 Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
CPU RAM Free: 12.07 GB


In [4]:
import torch
import psutil

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Usando dispositivo:', device)

if device.type == 'cuda':
    print("Nombre de la GPU:", torch.cuda.get_device_name(0))
    print("Memoria asignada:", round(torch.cuda.memory_allocated(0) / 1024**3, 1), 'GB')
    print("Memoria en caché:", round(torch.cuda.memory_reserved(0) / 1024**3, 1), 'GB')

print(f"RAM libre en CPU: {psutil.virtual_memory().available / 1024**3:.2f} GB")

Usando dispositivo: cuda
Nombre de la GPU: NVIDIA T1200 Laptop GPU
Memoria asignada: 0.0 GB
Memoria en caché: 0.0 GB
RAM libre en CPU: 11.92 GB


In [ ]:
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar el conjunto de datos
data = load_breast_cancer()
X, y = data.data, data.target

# Separar en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir a DMatrix (estructura optimizada para XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Configuración del modelo con aceleración por GPU
params = {
    'max_depth': 4,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'tree_method': 'hist',  # Utiliza la GPU
    'eval_metric': 'logloss',
    'device': 'cuda',  # Especifica el uso de GPU
}

# Entrenamiento del modelo
bst = xgb.train(params, dtrain, num_boost_round=100)

# Predicciones
y_pred_prob = bst.predict(dtest)
y_pred = (y_pred_prob > 0.5).astype(int)

# Evaluación
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo con GPU: {accuracy:.4f}")

Precisión del modelo con GPU: 0.9561


In [8]:
from lightgbm import LGBMRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Crear datos sintéticos
X, y = make_regression(n_samples=100000, n_features=20, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo con soporte GPU
model = LGBMRegressor(
    boosting_type='gbdt',
    device='gpu',                # ← activa el uso de GPU
    gpu_platform_id=0,           # ← opcional, si tienes más de una GPU
    gpu_device_id=0,             # ← opcional, para seleccionar GPU
    n_estimators=100,
    learning_rate=0.1
)

model.fit(X_train, y_train)

# Evaluación
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)
print(f"RMSE: {rmse:.4f}")


c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5100
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 20
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA T1200 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 20 dense feature groups (1.53 MB) transferred to GPU in 0.004244 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score -0.355959
RMSE: 434.5683


c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
